In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn 
import torch.optim as optim
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import optuna
# from optuna.trial import TrialState
import os
import joblib
import torch.nn.functional as F

In [3]:
start_list = [0,1,2,3,4,5,0]
number_of_corner_list = [1,2,3,4,5,6,6]
feature_list = [0, 1]
truncate_number = 60
for i in range(1):
    feature = feature_list[i]
    for i in range(1):
        start = start_list[i]
        number_of_corner = number_of_corner_list[i]
        df_begin = []
        num_row = []
        num_row_corner = []
        percent = []
    #     start = 0s
    #     number_of_corner = 1
        f_1 = 'beginner_expert_processedData/beginner/beginner_'
        f_3 = '.csv'
        num_begin = 19
#         curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1212.3, 1437.1]]
        curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]
#         curveList = [[103.9, 156.6], [156.6, 209.3], [316.6, 358.1], [358.1, 399.6], 
#                      [425.3, 471.6], [471.6, 517.9],p[590.5, 673.7], [673.7, 756.9],
#                      [1048.7, 1079.6], [1079.6, 1110.5], [1212.3, 1324.7], [1324.7, 1437.1]]

        df_concat = pd.DataFrame()
        for curve_num in range(start,number_of_corner):
            for idx in range(1, num_begin+1):
                tmp_file = f_1+str(idx)+'_new2'+f_3
                df = pd.read_csv(tmp_file)
                df = df.dropna()
                tmp = df.astype(float)
                tmp['level'] =0
                tmp['curve_number'] = curve_num
                tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
                num_row.append(np.size(tmpcorner,0)) 
                num_row_corner.append(np.size(tmpcorner,0)) 
                df_begin.append(tmpcorner)
                df_concat = pd.concat([df_concat,df_begin[idx-1]])      
            df_concat.to_csv('cornerData2/corner_'+str(curve_num+1)+'_begin'+'.csv')
            df_concat = pd.DataFrame()
            df_begin = []
        df_exp = []
        f_1 = 'beginner_expert_processedData/expert/expert_'
        f_3 = '.csv'
        num_exp = 19
        
        
        df_concat = pd.DataFrame()
        for curve_num in range(start,number_of_corner):
            for idx in range(1, num_exp+1):
                tmp_file = f_1+str(idx)+'_new2'+f_3
                df = pd.read_csv(tmp_file)
                df = df.dropna()
                tmp = df.astype(float)
                tmp['level'] =1
                tmp['curve_number'] = curve_num
                tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
                num_row.append(np.size(tmpcorner,0))
                num_row_corner.append(np.size(tmpcorner,0)) 
                df_exp.append(tmpcorner)
                df_concat = pd.concat([df_concat,df_exp[idx-1]])
            df_concat.to_csv('cornerData2/corner_'+str(curve_num+1)+'_expert'+'.csv')
            df_concat = pd.DataFrame()
            df_exp = []
            num_row_corner = np.array(num_row_corner)
            per = 60  ##truncate to 60
            percent.append(per)
            num_row_corner = []
        sequence_length = min(num_row)
        if sequence_length < truncate_number:
            sequnce_length = truncate_number + 5
        mean_row = round(np.mean(num_row))
        mean_row = min(num_row)

        if feature == 0:
            left_column = [
                'curve_number',
                'GPS Latitude','GPS Longitude',
                'CG Distance',
                'Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
            'Damper Velocity (Calc) RR','Corr Speed','Brake Pos',
            'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
            'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
            'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
            'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
            'Self Align Torque RL','Self Align Torque RR','Steering Angle','Suspension Travel FL','Suspension Travel FR',
            'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
            'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
            'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
            'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
            'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
            'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
            'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
            'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
            'Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration','level']

        else:
            left_column = ['curve_number','Brake Pos', 'Ground Speed', 'Steering Angle', 'Throttle Pos', 'Chassis Yaw Rate', 'Chassis Velocity X',
                               'Chassis Velocity Y','Chassis Velocity Z','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration',
                               'CG Distance',
                           'level']

        #Hyper-parameters
        num_epochs = 50
        batches = 1
        learning_rate = 0.001
        input_size = len(left_column)-1 # left column except 'level'
        output_size = 2 # Expert and Beginner
        hidden_size = 100 # ?
        num_layers = 2
        num_begin_train = round(num_begin*0.60)*(number_of_corner-start)
        num_exp_train = round(num_exp*0.60)*(number_of_corner-start)
        num_begin_test = num_begin*(number_of_corner-start) - num_begin_train
        num_exp_test = num_exp*(number_of_corner-start) - num_exp_train
        aug = 1

        
        class GRU(nn.Module):
            def __init__(self, input_size, hidden_size, num_layer, output_size):
                super(GRU, self).__init__()
                self.num_layers = num_layers
                self.hidden_size = hidden_size
                self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
                self.fc = nn.Linear(hidden_size, output_size)

            def forward(self, x):
                h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
                c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
                out, _ = self.gru(x, h0)
                out = out[:, -1, :]
                out = self.fc(out)
                return out
        

        ## Data Processing
        array_x = []
        array_y = []
        input_x = []
        input_y = []
        n_row = []

        df_tmp_begin = pd.DataFrame() 
        df_tmp_exp = pd.DataFrame() 
        for curve_num in range(start,number_of_corner):
            df_tmp_begin = pd.concat([df_tmp_begin,pd.read_csv('cornerData2/corner_'+str(curve_num+1)+'_begin.csv')])
            df_tmp_exp   = pd.concat([df_tmp_exp,pd.read_csv('cornerData2/corner_'+str(curve_num+1)+'_expert.csv')])    
        df_curve1 = pd.concat([df_tmp_begin, df_tmp_exp], ignore_index=True) 
        df_curve1 = df_curve1.loc[:,left_column]
        df_curve1_saved = df_curve1.loc[:,left_column] # data backup
        df_curve1.to_csv('cornerData2/corner_'+'_dfcurve1'+'.csv')
        datum = df_curve1_saved
        yyy = datum.pop('level')
        left = left_column.remove('level')
        for i in range(0,num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start)):
            y = yyy.loc[0:num_row[i]-1]
            x_original = datum.loc[0:num_row[i]-1]
            scaler = MinMaxScaler()
            x_normal = scaler.fit_transform(x_original)
            x_normal = scaler.transform(x_original)
            x_normal = np.pad(x_normal,[(0,60),(0,0)],'edge') #post padding ## edge padding
            x = pd.DataFrame(x_normal,columns=left)
            p = i//(num_begin*2)
            x = x.truncate(after=percent[p]-1)
            datum.drop(range(0,num_row[i]),inplace=True)
            datum.reset_index(drop=True, inplace=True)
            yyy.drop(range(0,num_row[i]),inplace=True)
            yyy.reset_index(drop=True, inplace=True)
            array_x.append(x)
            array_y.append(y)



        ## Randomize sequence 
        # sequence = np.arange((num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug/2)
        seq_train_begin = np.arange(num_begin_train)
        seq_test_begin = np.arange(num_begin_test) + num_begin_train
        seq_train_exp = seq_train_begin + num_begin*(number_of_corner-start)
        seq_test_exp = seq_test_begin + num_begin*(number_of_corner-start)
        print(seq_train_begin,seq_train_exp)
        np.random.seed(13)
        np.random.shuffle(seq_train_begin)
        np.random.seed(13)
        np.random.shuffle(seq_test_begin)
        np.random.seed(13)
        np.random.shuffle(seq_train_exp)
        np.random.seed(13)
        np.random.shuffle(seq_test_exp)


        sequence = np.concatenate((seq_train_begin, seq_train_exp, seq_test_begin, seq_test_exp), axis=None)
        sequence = sequence.astype('int')
        print(sequence)


        for i in sequence:
            input_x.append(array_x[i])
            input_y.append(array_y[i])
            p = i//(num_begin*2)
            n_row = n_row + [percent[p]]
        
    
            
        def train_mnist(trial):
            
            d2 = {'test sample': [], 'loss': [], 'accuracy': [], 'epoch' : [], 'batch':[],'learning rate' :[], 'hidden size':[],'hidden_layer':[]}
            df2 = pd.DataFrame(data=d2)
            loss_list = []
            iteration_list = []
            accuracy_list = []
            test_list=[]
            accuracy2_list=[]
            count = 0
            cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
              'train_batch_size' : 64,
              'test_batch_size' : 1000,
              'n_epochs' : 1,
              'seed' : 0,
              'log_interval' : 100,
              'save_model' : False,
              'lr'       : 0.0008674236780492814,          
              'hidden_size' : 126,
              'lr'       : trial.suggest_loguniform('lr', 5e-3, 5e-2),          
              'hidden_size' : trial.suggest_int('hidden_size', 60, 200),
              'optimizer': optim.Adam,
              'activation': F.relu}
        
            torch.manual_seed(cfg['seed'])
        
            gru = GRU(input_size, cfg['hidden_size'], num_layers, output_size)
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)       
#             optimizer = cfg['optimizer'](gru.parameters(), lr=cfg['lr'])
    
    
            for epoch in range(num_epochs):
                for i in range(0,(num_begin_train + num_exp_train)*aug):
                    # array type (numpy) 앞
                    X = np.array(input_x[i])
                    X = X.reshape(-1,n_row[i],input_size)
                    Y = np.array(input_y[i])

                    # tensor type (pytorch)
                    X = torch.from_numpy(X)
                    X = X.float()
                    Y = torch.tensor([Y[0]])
                    Y = Y.type(torch.LongTensor)
                    optimizer.zero_grad()
                    output = gru(X)
                    loss = criterion(output, Y)

                    # Backward and optimize
            #         optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                count += 1
                loss_list.append(loss.data)
                iteration_list.append(count)
                print (f'Epoch: [{epoch}/{num_epochs}]' f'Loss: {loss.item():.4f}')
            ## Test
                with torch.no_grad():
                    n_correct = 0
                    n_correct2 = 0
                    n_samples = 0
                    n_samples2 = 0

                    for i in range((num_begin_train + num_exp_train)*aug, (num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug-4):

                        # array type (numpy)
                        X = np.array(input_x[i])
                        X = X.reshape(-1,n_row[i],input_size)
                        Y = np.array(input_y[i])

                        # tensor type (pytorch)
                        X = torch.from_numpy(X)
                        X = X.float()
                        Y = torch.tensor([Y[0]])
                        Y = Y.type(torch.LongTensor)
                        output = gru(X)
                        _, predicted = torch.max(output.data, 1)
                        n_samples += Y.size(0)
                        n_correct += (predicted == Y).sum().item()

    
                    for i in range((num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug-4,(num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug):

                        # array type (numpy)
                        X = np.array(input_x[i])
                        X = X.reshape(-1,n_row[i],input_size)
                        Y = np.array(input_y[i])

                        # tensor type (pytorch)
                        X = torch.from_numpy(X)
                        X = X.float()
                        Y = torch.tensor([Y[0]])
                        Y = Y.type(torch.LongTensor)
                        output = gru(X)
                        _, predicted2 = torch.max(output.data, 1)
                        n_samples2 += Y.size(0)
                        n_correct2 += (predicted2 == Y).sum().item()
    
    
                    acc = 100.0 * n_correct / n_samples
                    acc2 = 100.0 * n_correct2 / n_samples2
                    accuracy_list.append(acc)
                    accuracy2_list.append(acc2)
                    print(f'Accuracy of the network on the {(num_begin_test + num_exp_test)*aug} test images: {acc} %')          
            plt.plot(iteration_list,loss_list)
            plt.xlabel("Number of iteration")
            plt.ylabel("Loss")
            plt.title("GRU: Loss vs Number of iteration")

            if feature == 0:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_allFeat1{curve_num}.png')
                plt.clf()
            else:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_selectedFeat1{curve_num}.png')
                plt.clf()

            plt.plot(iteration_list,accuracy_list)
            plt.xlabel("Number of iteration")
            plt.ylabel(" Validation Accuracy")
            plt.title("GRU: Validaiton Accuracy vs Number of iteration")

            if feature == 0:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_allFeat1_acc{curve_num}.png')
                plt.clf()
            else:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_selectedFeat1_acc{curve_num}.png')
                plt.clf()    

            plt.plot(iteration_list,accuracy2_list)
            plt.xlabel("Number of iteration")
            plt.ylabel(" Test Accuracy")
            plt.title("GRU: Test Accuracy vs Number of iteration")

            if feature == 0:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_allFeat1_acc{curve_num}.png')
                plt.clf()
            else:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_selectedFeat1_acc{curve_num}.png')
                plt.clf()  

            print(f'Accuracy of the network on the {num_begin_test + num_exp_test} test images: {acc} %')
            print(f'Loss: {loss.item():.4f}')
            return acc

        study = optuna.create_study(direction='maximize')
        study.optimize(train_mnist, n_trials=10)
        joblib.dump(study, f'cornerData2/mnist_optuna{curve_num}.pkl')
    
        study = joblib.load(f'cornerData2/mnist_optuna{curve_num}.pkl')
        df3 = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
        print(df3)






    

[I 2021-06-19 22:31:11,273] A new study created in memory with name: no-name-388d4187-0d35-406e-bebe-1c446728bebd


[ 0  1  2  3  4  5  6  7  8  9 10] [19 20 21 22 23 24 25 26 27 28 29]
[ 3  5  7  8  1  4 10  6  9  0  2 22 24 26 27 20 23 29 25 28 19 21 12 15
 14 16 17 18 11 13 31 34 33 35 36 37 30 32]
Epoch: [0/50]Loss: 0.9657
Accuracy of the network on the 16 test images: 66.66666666666667 %
Epoch: [1/50]Loss: 0.6945
Accuracy of the network on the 16 test images: 75.0 %
Epoch: [2/50]Loss: 0.6143
Accuracy of the network on the 16 test images: 50.0 %
Epoch: [3/50]Loss: 0.5902
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [4/50]Loss: 0.5369
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [5/50]Loss: 0.4466
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [6/50]Loss: 0.3146
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [7/50]Loss: 0.1638
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [8/50]Loss: 0.0668
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [9/50]Loss: 0.0241
Accuracy of the network on the 16 te

[I 2021-06-19 22:31:40,696] Trial 0 finished with value: 83.33333333333333 and parameters: {'lr': 0.005829218972571184, 'hidden_size': 96}. Best is trial 0 with value: 83.33333333333333.


Accuracy of the network on the 16 test images: 83.33333333333333 %
Loss: 0.0002
Epoch: [0/50]Loss: 0.4850
Accuracy of the network on the 16 test images: 33.333333333333336 %
Epoch: [1/50]Loss: 0.5923
Accuracy of the network on the 16 test images: 33.333333333333336 %
Epoch: [2/50]Loss: 0.6289
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [3/50]Loss: 0.5793
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [4/50]Loss: 0.5212
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [5/50]Loss: 0.4332
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [6/50]Loss: 0.3067
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [7/50]Loss: 0.1706
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [8/50]Loss: 0.0743
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [9/50]Loss: 0.0256
Accuracy of the network on the 16 test images: 91.66666

[I 2021-06-19 22:32:19,272] Trial 1 finished with value: 91.66666666666667 and parameters: {'lr': 0.015560956009423973, 'hidden_size': 156}. Best is trial 1 with value: 91.66666666666667.


...

[I 2021-06-19 22:36:19,049] Trial 8 finished with value: 83.33333333333333 and parameters: {'lr': 0.009682464517731587, 'hidden_size': 109}. Best is trial 4 with value: 100.0.


Accuracy of the network on the 16 test images: 83.33333333333333 %
Loss: 0.0002
Epoch: [0/50]Loss: 1.2086
Accuracy of the network on the 16 test images: 66.66666666666667 %
Epoch: [1/50]Loss: 0.6735
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [2/50]Loss: 0.6058
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [3/50]Loss: 0.5706
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [4/50]Loss: 0.5080
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [5/50]Loss: 0.4159
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [6/50]Loss: 0.2921
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [7/50]Loss: 0.1640
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [8/50]Loss: 0.0713
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [9/50]Loss: 0.0297
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [10/50]Loss: 0.0163
Accuracy of the network on the 16 test images: 91.666666666

[I 2021-06-19 22:36:44,559] Trial 9 finished with value: 83.33333333333333 and parameters: {'lr': 0.0390957419888935, 'hidden_size': 61}. Best is trial 4 with value: 100.0.


Accuracy of the network on the 16 test images: 83.33333333333333 %
Loss: 0.0003
   number       value               duration  params_hidden_size  params_lr
0       0   83.333333 0 days 00:00:29.420538                  96   0.005829
1       1   91.666667 0 days 00:00:38.573323                 156   0.015561
2       2   91.666667 0 days 00:00:35.784607                 143   0.033787
3       3   91.666667 0 days 00:00:28.912149                  92   0.007693
4       4  100.000000 0 days 00:00:42.927380                 198   0.008240
5       5  100.000000 0 days 00:00:35.142250                 141   0.024455
6       6   91.666667 0 days 00:00:37.442424                 164   0.033066
7       7   91.666667 0 days 00:00:28.664602                  90   0.005169
8       8   83.333333 0 days 00:00:30.899161                 109   0.009682
9       9   83.333333 0 days 00:00:25.509172                  61   0.039096


<Figure size 432x288 with 0 Axes>